In [ ]:
import nglview as nv
import ipywidgets as widgets

# Load a PDB file
pdb_file = "assets/5ceo.pdb"


view = nv.show_structure_file(pdb_file)
view.representations = [
    {"type": "cartoon", "params": {
        "sele": "protein", "color": "blue"
    }},
    {"type": "ball+stick", "params": {
        "sele": "hetero"
    }}
]
# view = nv.show_pdbid("5ceo")
# view.clear_representations()
# view.remove_cartoon()
# view.add_cartoon(selection="protein", color='blue')
# view.add_surface(selection='protein', opacity=0.001)
view.add_representation("ball+stick", selection="LIG")
view


In [ ]:
# Cycle through pockets in NGLView
from nanome.api.structure import Complex
import os
import time

def view_pocket(view, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    view.remove_surface()
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])
    view.add_surface(selection=pocket_res_nums, color='red', opacity=0.5)
    return True

fpocket_out = 'assets/5ceo_fpocket_out'
view_pocket(view, fpocket_out, 1)

# cycle_pocket_surfaces(view, fpocket_out)

In [ ]:
def cycle_pocket_surfaces(view, fpocket_out: str):
    # Clear all other surface representations
    i = 1
    while True:
        view.remove_surface()
        pocket_file = f'{fpocket_out}/pockets/pocket{i}_atm.pdb'
        if not os.path.exists(pocket_file):
            break
        success = view_pocket(view, fpocket_out, i)
        if not success:
            break
        print(f"Pocket {i}")
        time.sleep(1)
        i += 1

view
cycle_pocket_surfaces(view, fpocket_out)

In [ ]:
# Set up nanome plugin
import os
from nanome.beta.redis_interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = "4TOGN"  # Use room id code you receive when activating cookbook plugin

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

In [ ]:
plugin.send_files_to_load([pdb_file])

In [ ]:
comp = plugin.request_workspace().complexes[-1]

In [ ]:
from nanome.util import Color

def render_pocket_surface(comp, fpocket_out, pocket_num):
    pocket_file = f'{fpocket_out}/pockets/pocket{pocket_num}_atm.pdb'
    if not os.path.exists(pocket_file):
        print(f"Pocket {pocket_num} not found")
        return False
    pocket_comp = Complex.io.from_pdb(path=pocket_file)
    pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])
    
    comp.remarks = pocket_comp.remarks
    breakpoint()
    for comp_mol, pocket_mol in zip(comp.molecules, pocket_comp.molecules):
        comp_mol.associateds = pocket_mol.associateds
    for res in comp.residues:
        surface_color = Color.Red() if res.serial in pocket_res_nums else Color.Blue()
        for atom in res.atoms:
            atom.surface_rendering = True
            atom.surface_color = surface_color
    comp._surface_dirty = True

pocket_number = 2
render_pocket_surface(comp, fpocket_out, pocket_number)
plugin.update_structures_deep([comp])


In [ ]:
# THIS CELL ISN'T WORKING YET.

from nanome.api.structure import Complex, Molecule
import copy

def setup_fpocket_complex(pdb_file, fpocket_out):
    new_comp = Complex()
    new_comp._molecules = []

    full_comp_copy = Complex.io.from_pdb(path=pdb_file)
    full_mol = next(full_comp_copy.molecules)

    i = 1
    while True:
        pocket_file = f'{fpocket_out}/pockets/pocket{i}_atm.pdb'
        if i > 1 or not os.path.exists(pocket_file):
            break
        print(f"Pocket {i}")
        
        # Get pocket residue serials from pocket_file
        pocket_comp = Complex.io.from_pdb(path=pocket_file)
        pocket_mol = next(pocket_comp.molecules)
        pocket_res_nums = sorted([res.serial for res in pocket_comp.residues])

        new_mol = Molecule()
        for ch in full_mol.chains:
            new_mol.add_chain(ch)
        
        # Color pocket residues red, and non pocket residues blue
        # for res in new_mol.residues:
        #     surface_color = Color.Red() if res.serial in pocket_res_nums else Color.Blue()
        #     for atom in res.atoms:
        #         atom.surface_rendering = True
        #         atom.surface_color = surface_color
        # full_comp_copy.remove_molecule(full_mol)
        new_comp.add_molecule(new_mol)
        i += 1
    return new_comp

fpocket_comp = setup_fpocket_complex(pdb_file, fpocket_out)
plugin.update_structures_deep([fpocket_comp])